**АНАЛИЗ БАЗЫ ДАННЫХ СЕРВИСА ДЛЯ ЧТЕНИЯ КНИГ ПО ПОДПИСКЕ**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Оглавление:**


[Шаг 1. Изучение таблиц](#step_1)


[Шаг 2. Cколько книг вышло после 1 января 2000 года?](#step_2)


[Шаг 3. Количество обзоров и среднюю оценку для каждой книги](#step_3)


[Шаг 4. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц](#step_4)
   

[Шаг 5. Определим автора с самой высокой средней оценкой книг (у которых было 50 и более оценок)](#step_5)


[Шаг 6. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок](#step_5)

In [2]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

<a id='step_2'></a>
**Шаг №1. Изучение таблиц**

In [3]:
#посмотрим на таблицы
query = '''
SELECT *
FROM orders
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,id,buyts,revenue,uid
0,1,2017-06-01 00:10:00,$17.00,10329302124590727494
1,2,2017-06-01 00:25:00,$0.55,11627257723692907447
2,3,2017-06-01 00:27:00,$0.37,17903680561304213844
3,4,2017-06-01 00:29:00,$0.55,16109239769442553005
4,5,2017-06-01 07:58:00,$0.37,14200605875248379450


In [4]:
query = '''
SELECT *
FROM books
LIMIT 5
'''

pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Книга Memoirs of a Geisha автора

In [5]:
query = '''
SELECT books.title, authors.author as autor, COUNT(title) as cnt
FROM books JOIN authors ON authors.author_id = books.author_id
GROUP BY title, author
HAVING COUNT(title) > 1
'''

pd.io.sql.read_sql(query, con = engine) 

,title,autor,cnt
0,Memoirs of a Geisha,Arthur Golden,2


In [6]:
query = '''
SELECT 
    books.title AS book_name, 
    publishers.publisher AS publisher
FROM 
    books 
    INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE title = 'Memoirs of a Geisha'
'''

pd.io.sql.read_sql(query, con = engine) 

,book_name,publisher
0,Memoirs of a Geisha,Random House Large Print Publishing
1,Memoirs of a Geisha,Vintage Books USA


In [7]:
query = '''
SELECT COUNT(title), COUNT(DISTINCT title)
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,count,count
0,1000,999


Всего уникальных 999 книг, но одна книга Memoirs of a Geisha автора Arthur Golden была выпущена двумя разными издательствами.

In [8]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [9]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [10]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
query = '''
SELECT AVG(rating)
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,avg
0,3.928284


In [12]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<a id='step_2'></a>
**Шаг №2. Посчитаем, сколько книг вышло после 1 января 2000 года**

In [13]:
#query = '''
#SHOW COLUMNS 
#FROM reviews
#'''
#pd.io.sql.read_sql(query, con = engine) 

In [14]:
query = '''
SELECT COUNT(*) AS cnt
FROM books
WHERE publication_date >= '2000-01-01'

'''
pd.io.sql.read_sql(query, con = engine) 

,cnt
0,821


In [15]:
query = '''
SELECT MIN(publication_date), MAX(publication_date)
FROM books
WHERE publication_date >= '2000-01-01'

'''
pd.io.sql.read_sql(query, con = engine) 

,min,max
0,2000-01-01,2020-03-31


После 01-01-2020 (вкл.) было вышло 821 книг. 

<a id='step_3'></a>
**Шаг №3. Для каждой книги посчитаем количество обзоров и среднюю оценку**

In [63]:
#query = '''
#SELECT   
#    books.title AS book_name, 
#    authors.author AS author, 
#    AVG(ratings.rating) AS rating_avg, 
#    COUNT(reviews.review_id) AS reviews_cnt
    
#FROM 
#    books 
#    LEFT JOIN authors ON authors.author_id = books.author_id
#    LEFT JOIN ratings ON ratings.book_id = books.book_id
#    LEFT JOIN reviews ON reviews.book_id = books.book_id

#GROUP BY book_name,  author 
#ORDER BY reviews_cnt DESC;
#'''

#pd.io.sql.read_sql(query, con = engine) 

In [64]:
query = '''

SELECT books.title AS book_name, 
       authors.author AS author, 
       ratings_mean.avg_rating AS avg_rating, 
       reviews_count.avg_reviews AS avg_reviews
FROM (
    SELECT   
        COUNT(reviews.text) AS avg_reviews, 
        reviews.book_id AS book_id
    FROM reviews 
    GROUP BY reviews.book_id) AS reviews_count 
    FULL OUTER JOIN (
        SELECT   
            AVG(ratings.rating) AS avg_rating, 
            ratings.book_id AS book_id    
        FROM ratings
        GROUP BY ratings.book_id
        ) AS ratings_mean ON ratings_mean.book_id = reviews_count.book_id
    INNER JOIN books ON books.book_id = reviews_count.book_id
    INNER JOIN authors ON authors.author_id = books.author_id
ORDER BY avg_rating DESC;


'''
pd.io.sql.read_sql(query, con = engine) 

,book_name,author,avg_rating,avg_reviews
0,Wherever You Go There You Are: Mindfulness Me...,Jon Kabat-Zinn,5.00,2
1,Piercing the Darkness (Darkness #2),Frank E. Peretti,5.00,2
2,The Walking Dead Book One (The Walking Dead #...,Robert Kirkman/Tony Moore/Charlie Adlard/Cliff...,5.00,2
3,Captivating: Unveiling the Mystery of a Woman'...,John Eldredge/Stasi Eldredge,5.00,2
4,Arrows of the Queen (Heralds of Valdemar #1),Mercedes Lackey,5.00,2
...,...,...,...,...
989,The World Is Flat: A Brief History of the Twen...,Thomas L. Friedman,2.25,3
990,Junky,William S. Burroughs/Oliver Harris/Allen Gins...,2.00,2
991,His Excellency: George Washington,Joseph J. Ellis,2.00,2
992,Drowning Ruth,Christina Schwarz,2.00,3


<a id='step_4'></a>
**Шаг №4. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц**

In [16]:
query = '''
SELECT   
    publishers.publisher AS publisher,
    COUNT(books.title) as books_total
FROM 
    books 
    INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY publisher
ORDER BY books_total DESC
LIMIT 10;

'''
pd.io.sql.read_sql(query, con = engine) 

,publisher,books_total
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


Издательство Penguin Books выпустила наибольшее кол-во книг в размере 42 ед. толще 50 страниц. 

<a id='step_5'></a>
**Шаг 5. Определим автора с самой высокой средней оценкой книг (у которых было 50 и более оценок).**

In [103]:
"""query = '''
SELECT   
    authors.author AS author, 
    AVG(ratings.rating) AS rating_avg,
    COUNT(ratings.rating) AS rating_cnt
    
FROM 
    books 
    LEFT JOIN authors ON authors.author_id = books.author_id
    LEFT JOIN ratings ON ratings.book_id = books.book_id
    
GROUP BY author
HAVING COUNT(ratings.rating) >= 50
ORDER BY rating_avg DESC

'''
pd.io.sql.read_sql(query, con = engine) """

"query = '''\nSELECT   \n    authors.author AS author, \n    AVG(ratings.rating) AS rating_avg,\n    COUNT(ratings.rating) AS rating_cnt\n    \nFROM \n    books \n    LEFT JOIN authors ON authors.author_id = books.author_id\n    LEFT JOIN ratings ON ratings.book_id = books.book_id\n    \nGROUP BY author\nHAVING COUNT(ratings.rating) >= 50\nORDER BY rating_avg DESC\n\n'''\npd.io.sql.read_sql(query, con = engine) "

Самая высокая оценка у авторов Diana Gabaldon, Джоан Роулинг, Агата Кристи, Markus Zusak и Толкиена (при мин. кол-во оценок 50). Но стоит выделить Джоан Роулинг: у нее самая высокая оценка и наибольшее кол-во отзывов.

In [118]:
# Определим автора с самой высокой средней оценкой книг (у которых было 50 и более оценок для одной книги).
query = '''
SELECT books.title AS book_name, 
       authors.author AS author, 
       ratings_grouped.rating_avg AS avg_rating, 
       ratings_grouped.rating_cnt AS cnt_rating
FROM books 
    INNER JOIN (
        SELECT   
            book_id,
            AVG(ratings.rating) AS rating_avg,
            COUNT(ratings.rating) AS rating_cnt
        FROM ratings     
        GROUP BY ratings.book_id 
        HAVING COUNT(ratings.rating) > 50) AS ratings_grouped ON ratings_grouped.book_id = books.book_id
    
    INNER JOIN authors ON authors.author_id = books.author_id
ORDER BY ratings_grouped.rating_avg DESC
LIMIT 5;

'''
pd.io.sql.read_sql(query, con = engine) 

,book_name,author,avg_rating,cnt_rating
0,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.414634,82
1,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien,4.391892,74
2,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,4.287500,80
3,The Book Thief,Markus Zusak/Cao Xuân Việt Khương,4.264151,53
4,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.246575,73


In [119]:
# Определим автора с самой высокой средней оценкой книг (у которых было 50 и более оценок суммарно по автору).
query = '''
SELECT 
    authors.author AS author, 
    COUNT(ratings.rating) AS rating_cnt, 
    AVG(ratings.rating) AS rating_mean   
FROM ratings 
    LEFT JOIN books ON books.book_id = ratings.book_id
    INNER JOIN authors ON authors.author_id = books.author_id
GROUP BY authors.author     
HAVING COUNT(ratings.rating) > 50
ORDER BY AVG(ratings.rating) DESC
LIMIT 5; 

'''
pd.io.sql.read_sql(query, con = engine) 

,author,rating_cnt,rating_mean
0,J.K. Rowling/Mary GrandPré,312,4.288462
1,Agatha Christie,53,4.283019
2,Markus Zusak/Cao Xuân Việt Khương,53,4.264151
3,J.R.R. Tolkien,166,4.240964
4,Roald Dahl/Quentin Blake,62,4.209677


In [120]:
"""query = '''
SELECT 
    COUNT(ratings.rating) AS rating_cnt
FROM ratings 
    LEFT JOIN books ON books.book_id = ratings.book_id
    INNER JOIN authors ON authors.author_id = books.author_id
   

'''
pd.io.sql.read_sql(query, con = engine) """

"query = '''\nSELECT \n    COUNT(ratings.rating) AS rating_cnt\nFROM ratings \n    LEFT JOIN books ON books.book_id = ratings.book_id\n    INNER JOIN authors ON authors.author_id = books.author_id\n   \n\n'''\npd.io.sql.read_sql(query, con = engine) "

<a id='step_6'></a>
**Шаг 6. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок**

In [18]:
query = '''
SELECT   
   COUNT(reviews.review_id) / COUNT(DISTINCT reviews.username) AS avg_reviews
    
FROM 
   reviews

WHERE reviews.username IN (
    SELECT users_list.username

        FROM (
            SELECT   
                ratings.username AS username,
                COUNT(ratings.rating) AS ratings_cnt    
            FROM 
                ratings 
            GROUP BY ratings.username
            HAVING COUNT(ratings.rating) > 50) AS users_list)

'''
pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24


В среднем пользователи, которые оценки 50 и более раз, оставляют в среднем 24 обзора